In [83]:
import numpy as np
from qutip import*
import matplotlib.pyplot as plt
import math
import cmath 
pi=np.pi
n=200
N=20



In [89]:
# defining some variables and hamiltonian
w1 = 2*pi*1.34*10**6
w2 = 2*pi*1.47*10**6
alpha1 = 0
alpha2 = 0

# Qubit and oscillator operators
a1  = tensor(qeye(2),destroy(N),qeye(N))  # Oscillator  annihilation operator and identity on qubit
a2  = tensor(qeye(2),qeye(N),destroy(N)) 
sm = tensor( (basis(2,1)*basis(2,1).dag()) ,qeye(N),qeye(N) ) #Excited state projection for qubit and identity on mode

# Identity operators for qubit and oscillator
I_q = qeye(2)
I_o = qeye(N)
sm_q=basis(2,1)*basis(2,1).dag() #just the excited state projection for the qubit
#sm_q = sigmaz()
# free hamiltonian of the oscillators 
H_0 = w1*a1.dag()*a1 + w2*a2.dag()*a2
    
# interaction hamiltonian 
H_int11 = tensor(sm_q, destroy(N).dag(),qeye(N))
H_int21 = tensor(sm_q,destroy(N), qeye(N))
H_int12 = tensor(sm_q, qeye(N), destroy(N).dag())
H_int22 = tensor(sm_q, qeye(N), destroy(N))
proj_ground= tensor(fock_dm(2,0), qeye(N),qeye(N))
proj_excited = tensor(fock_dm(2,1),qeye(N),qeye(N))

In [90]:
def R_phi(phi):
    # Define the matrix elements based on the given formula
    matrix1 = (1 / np.sqrt(2)) * np.array([[1, np.exp(1j * phi)],
                                          [-np.exp(-1j *phi), 1]])
    matrix = tensor(Qobj(matrix1),qeye(N),qeye(N))
    # Return as a QuTiP quantum object
    return Qobj(matrix)

def interaction_int(w1,w2,lam1,lam2,d1,d2,ini_state,t_in,tau):    
    temp = t_in + tau
    H = QobjEvo( [[H_int11, lambda t, args:(lam1* np.exp(-1j * (w1 - d1) * t)*np.exp(1j*w1*t))],[H_int21, lambda t, args:(lam1.conjugate() * np.exp(1j * (w1 - d1) * t)*np.exp(-1j*w1*t))],[H_int12, lambda t, args:(lam2* np.exp(-1j * (w2 - d2) * t)*np.exp(1j*w2*t))],[H_int22, lambda t, args:(lam2.conjugate()* np.exp(1j * (w2 - d2) * t)*np.exp(-1j*w2*t))]])
    t= np.linspace(t_in,temp,n)
    fin_state = mesolve(H,ini_state,t,[],[])    
    return fin_state

def interaction_diftime(w1,w2,lam1,lam2,d1,d2,ini_state,t_in1,t_in2,tau):
    temp1 = t_in1 + tau
    temp2 = t_in2 + tau
    H = QobjEvo( [[H_int11, lambda t, args:(lam1* np.exp(-1j * (w1 - d1) * t)*np.exp(1j*w1*t))],[H_int21, lambda t, args:(lam1.conjugate()* np.exp(1j * (w1 - d1) * t)*np.exp(-1j*w1*t))],[H_int12, lambda t, args:(lam2* np.exp(-1j * (w2 - d2) * t)*np.exp(1j*w2*t))],[H_int22, lambda t, args:(lam2.conjugate()* np.exp(1j * (w2 - d2) * t)*np.exp(-1j*w2*t))]])
    H1 = QobjEvo( [[H_int11, lambda t, args:(lam1* np.exp(-1j * (w1 - d1) * t)*np.exp(1j*w1*t))],[H_int21, lambda t, args:(lam1.conjugate()* np.exp(1j * (w1 - d1) * t)*np.exp(-1j*w1*t))]])
    H2 = QobjEvo( [[H_int12, lambda t, args:(lam2* np.exp(-1j * (w2 - d2) * t)*np.exp(1j*w2*t))],[H_int22, lambda t, args:(lam2.conjugate()* np.exp(1j * (w2 - d2) * t)*np.exp(-1j*w2*t))]])
    if t_in1==t_in2:
        temp = temp1
        t= np.linspace(t_in1,temp,n)
        fin_state = mesolve(H,ini_state,t,[],[]) 
    if t_in2>t_in1:
        temp = t_in2
        #print(temp)
        t= np.linspace(t_in1,temp,n)
        fin_state = mesolve(H1,ini_state,t,[],[])
        t = np.linspace(t_in2,temp1,n)
        fin_state = mesolve(H,fin_state.final_state,t,[],[])
        t = np.linspace(temp1,temp2,n)
        fin_state = mesolve(H2,fin_state.final_state,t,[],[])
    if t_in1>t_in2:
        temp = t_in1
        t= np.linspace(t_in2,temp,n)
        fin_state = mesolve(H2,ini_state,t,[],[])
        temp = t_in2
        t = np.linspace(t_in1,temp2,n)
        fin_state = mesolve(H,fin_state.final_state,t,[],[])
        temp = t_in2 
        t = np.linspace(temp2,temp1,n)
        fin_state = mesolve(H1,fin_state.final_state,t,[],[])    
    return fin_state    
def ramsey(phi,w1,w2,lam1,lam2,d1,d2,ini_state,t_in1,t_in2,tau):
    state_after_1pulse = R_phi(phi)*ini_state
    interaction_time = interaction_diftime(w1,w2,lam1,lam2,d1,d2,state_after_1pulse,t_in1,t_in2,tau)
    second_pulse = R_phi(0)*interaction_time.final_state
    return second_pulse
def measure(state):
    p_plus = expect(proj_excited,state)
    p_minus = expect(proj_ground,state)
    state_e = (1/np.sqrt(p_plus))*proj_excited*state
    state_g = (1/np.sqrt(p_minus))*proj_ground*state

    return p_plus,p_minus,state_e,state_g


In [91]:
x=Qobj(np.array([[0, 1],[1, 0]]))
notgate=tensor(x,qeye(N),qeye(N))
t=110.67*10**-6
d=3.5*2*pi*10**3
l=2.5*10**3*2*pi#18.238*2*pi*10**3
t_b = 600*10**-6
l=l

In [92]:
# r1t1 = 0 
# r1t2 = math.ceil((t_b+t)/(r1t1 + (2*pi/d) ))*(r1t1 + (2*pi/d) )
# r1t3 = math.ceil((t_b+t)/(r1t2 + (3*pi/d)  ))*(r1t2 + (3*pi/d) )

r1t1 = 0 
r1t2 = (2*pi*math.ceil((t_b+r1t1+t)/(2*pi/d))/d)
r1t3 = pi/d + (2*pi*math.ceil((t_b+r1t2+t)/(2*pi/d))/d)

# r2t1 = 2*pi/(3*d) 
# r2t2 = ((2*pi/(3*d))) + (2*pi*math.ceil((t_b+r2t1+t_b)/(2*pi/d))/d)
# r2t3 = ((5*pi/(3*d))) + (2*pi*math.ceil((t_b+r2t2+t_b)/(2*pi/d))/d)

r2t1 = 2*pi/(3*d) 
r2t2 = ((2*pi/(3*d))) + (2*pi*math.ceil((t_b+r2t1+t)/(2*pi/d))/d)
r2t3 = ((5*pi/(3*d))) + (2*pi*math.ceil((t_b+r2t2+t)/(2*pi/d))/d)

r3t1 = 0
r3t2 = (2*pi*(math.ceil((t_b+r3t1+t)/(2*pi/d)))/d)
r3t3 = (2*pi*(math.ceil((t_b+r3t2+t)/(2*pi/d)))/d)




#If the columns have to start at the same time

c1t1= c2t1 = 0
c1t2 = c2t2 = (2*pi*math.ceil((t_b+c1t1+t_b)/(2*pi/d))/d)
c1t3 = c2t3 = (2*pi*math.ceil((t_b+c1t2+t_b)/(2*pi/d))/d)

c3t1 = pi/d
c3t2 = 5*pi/(3*d) + (2*pi*math.ceil((t_b+c3t1+t)/(2*pi/d))/d)
c3t3 = 7*pi/(3*d) + (2*pi*math.ceil((t_b+c3t2+t)/(2*pi/d))/d)



In [93]:
# Define the coherent states for the oscillator
coherent_state1 = coherent(N, alpha1)  # Coherent state for the oscillator here at vacuum
coherent_state2 = coherent(N,alpha2)

# Combine the states using the tensor product (qubit, mode 1, mode 2)
initial_state = tensor(basis(2,0),coherent_state1,coherent_state2)

def row1(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32):
    # Now we find the expectation value of the first term in the expresion (A11R A12R A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403
    print(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32)
    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R111 = ramsey(angle,w1,w2,l,0,d,d,initial_state,r1t11,r1t12,t)
    R111_prob = measure(R111) # +,- prob(t1 =+),Prob(t1=-)


    R112 = ramsey(angle,w1,w2,0,l,d,d,notgate*R111_prob[2],r1t21,r1t22,t)
    R112_prob = measure(R112) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R113 = ramsey(angle*2,w1,w2,l,l,d,d,notgate*R112_prob[2],r1t31,r1t32,t)
    R113_prob = measure(R113) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R114 = ramsey(angle*2,w1,w2,l,l,d,d,R112_prob[3],r1t31,r1t32,t)
    R114_prob = measure(R114) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R115 = ramsey(angle,w1,w2,0,l,d,d,R111_prob[3],r1t21,r1t22,t)
    R115_prob = measure(R115)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R116 = ramsey(angle*2,w1,w2,l,l,d,d, notgate*R115_prob[2],r1t31,r1t32,t)
    R116_prob = measure(R116) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R117 = ramsey(angle*2,w1,w2,l,l,d,d,R115_prob[3],r1t31,r1t32,t)
    R117_prob = measure(R117) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11I A12I A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R121 = ramsey(angle+pi/2,w1,w2,l,0,d,d,initial_state,r1t11,r1t12,t)
    R121_prob = measure(R121) # +,- prob(t1 =+),Prob(t1=-)


    R122 = ramsey(angle+pi/2,w1,w2,0,l,d,d,notgate*R121_prob[2],r1t21,r1t22,t)
    R122_prob = measure(R122) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R123 = ramsey(angle*2,w1,w2,l,l,d,d,notgate*R122_prob[2],r1t31,r1t32,t)
    R123_prob = measure(R123) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R124 = ramsey(angle*2,w1,w2,l,l,d,d,R122_prob[3],r1t31,r1t32,t)
    R124_prob = measure(R124) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R125 = ramsey(angle+pi/2,w1,w2,0,l,d,d,R121_prob[3],r1t21,r1t22,t)
    R125_prob = measure(R125)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R126 = ramsey(angle*2,w1,w2,l,l,d,d, notgate*R125_prob[2],r1t31,r1t32,t)
    R126_prob = measure(R126) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R127 = ramsey(angle*2,w1,w2,l,l,d,d,R125_prob[3],r1t31,r1t32,t)
    R127_prob = measure(R127) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    # Now we find the expectation value of the first term in the expresion (A11I A12R A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R131 = ramsey(angle+pi/2,w1,w2,l,0,d,d,initial_state,r1t11,r1t12,t)
    R131_prob = measure(R131) # +,- prob(t1 =+),Prob(t1=-)


    R132 = ramsey(angle,w1,w2,0,l,d,d,notgate*R131_prob[2],r1t21,r1t22,t)
    R132_prob = measure(R132) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R133 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,notgate*R132_prob[2],r1t31,r1t32,t)
    R133_prob = measure(R133) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R134 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R132_prob[3],r1t31,r1t32,t)
    R134_prob = measure(R134) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R135 = ramsey(angle,w1,w2,0,l,d,d,R131_prob[3],r1t21,r1t22,t)
    R135_prob = measure(R135)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R136 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d, notgate*R135_prob[2],r1t31,r1t32,t)
    R136_prob = measure(R136) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R137 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R135_prob[3],r1t31,r1t32,t)
    R137_prob = measure(R137) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11R A12I A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    R141 = ramsey(angle,w1,w2,l,0,d,d,initial_state,r1t11,r1t12,t)
    R141_prob = measure(R141) # +,- prob(t1 =+),Prob(t1=-)


    R142 = ramsey(angle+pi/2,w1,w2,0,l,d,d,notgate*R141_prob[2],r1t21,r1t22,t)
    R142_prob = measure(R142) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R143 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,notgate*R142_prob[2],r1t31,r1t32,t)
    R143_prob = measure(R143) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R144 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R142_prob[3],r1t31,r1t32,t)
    R144_prob = measure(R144) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R145 = ramsey(angle+pi/2,w1,w2,0,l,d,d,R141_prob[3],r1t21,r1t22,t)
    R145_prob = measure(R145)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R146 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d, notgate*R145_prob[2],r1t31,r1t32,t)
    R146_prob = measure(R146) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R147 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R145_prob[3],r1t31,r1t32,t)
    R147_prob = measure(R147) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    #three time corellation
    expectterm1= (R111_prob[0]*R112_prob[0]*(R113_prob[0]-R113_prob[1])) + (R111_prob[0]*R112_prob[1]*(-R114_prob[0]+R114_prob[1])) + (R111_prob[1]*R115_prob[0]*(-R116_prob[0]+R116_prob[1]))+(R111_prob[1]*R115_prob[1]*(R117_prob[0]-R117_prob[1]))
    expectterm2=(R121_prob[0]*R122_prob[0]*(R123_prob[0]-R123_prob[1])) + (R121_prob[0]*R122_prob[1]*(-R124_prob[0]+R124_prob[1])) + (R121_prob[1]*R125_prob[0]*(-R126_prob[0]+R126_prob[1]))+(R121_prob[1]*R125_prob[1]*(R127_prob[0]-R127_prob[1]))
    expectterm3=(R131_prob[0]*R132_prob[0]*(R133_prob[0]-R133_prob[1])) + (R131_prob[0]*R132_prob[1]*(-R134_prob[0]+R134_prob[1])) + (R131_prob[1]*R135_prob[0]*(-R136_prob[0]+R136_prob[1]))+(R131_prob[1]*R135_prob[1]*(R137_prob[0]-R137_prob[1]))
    expectterm4=(R141_prob[0]*R142_prob[0]*(R143_prob[0]-R143_prob[1])) + (R141_prob[0]*R142_prob[1]*(-R144_prob[0]+R144_prob[1])) + (R141_prob[1]*R145_prob[0]*(-R146_prob[0]+R146_prob[1]))+(R141_prob[1]*R145_prob[1]*(R147_prob[0]-R147_prob[1]))

    total=  (expectterm1 - expectterm2 - expectterm3 - expectterm4)
    
    return total




 def row2(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32):
    # Now we find the expectation value of the first term in the expresion (A11R A12R A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403
    print(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32)
    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R111 = ramsey(angle,w1,w2,0,l,d,d,initial_state,r1t11,r1t12,t)
    R111_prob = measure(R111) # +,- prob(t1 =+),Prob(t1=-)


    R112 = ramsey(angle,w1,w2,l,0,d,d,notgate*R111_prob[2],r1t21,r1t22,t)
    R112_prob = measure(R112) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R113 = ramsey(angle*2,w1,w2,l,l,d,d,notgate*R112_prob[2],r1t31,r1t32,t)
    R113_prob = measure(R113) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R114 = ramsey(angle*2,w1,w2,l,l,d,d,R112_prob[3],r1t31,r1t32,t)
    R114_prob = measure(R114) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R115 = ramsey(angle,w1,w2,l,0,d,d,R111_prob[3],r1t21,r1t22,t)
    R115_prob = measure(R115)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R116 = ramsey(angle*2,w1,w2,l,l,d,d, notgate*R115_prob[2],r1t31,r1t32,t)
    R116_prob = measure(R116) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R117 = ramsey(angle*2,w1,w2,l,l,d,d,R115_prob[3],r1t31,r1t32,t)
    R117_prob = measure(R117) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11I A12I A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R121 = ramsey(angle+pi/2,w1,w2,0,l,d,d,initial_state,r1t11,r1t12,t)
    R121_prob = measure(R121) # +,- prob(t1 =+),Prob(t1=-)


    R122 = ramsey(angle+pi/2,w1,w2,l,0,d,d,notgate*R121_prob[2],r1t21,r1t22,t)
    R122_prob = measure(R122) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R123 = ramsey(angle*2,w1,w2,l,l,d,d,notgate*R122_prob[2],r1t31,r1t32,t)
    R123_prob = measure(R123) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R124 = ramsey(angle*2,w1,w2,l,l,d,d,R122_prob[3],r1t31,r1t32,t)
    R124_prob = measure(R124) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R125 = ramsey(angle+pi/2,w1,w2,l,0,d,d,R121_prob[3],r1t21,r1t22,t)
    R125_prob = measure(R125)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R126 = ramsey(angle*2,w1,w2,l,l,d,d, notgate*R125_prob[2],r1t31,r1t32,t)
    R126_prob = measure(R126) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R127 = ramsey(angle*2,w1,w2,l,l,d,d,R125_prob[3],r1t31,r1t32,t)
    R127_prob = measure(R127) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    # Now we find the expectation value of the first term in the expresion (A11I A12R A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R131 = ramsey(angle+pi/2,w1,w2,0,l,d,d,initial_state,r1t11,r1t12,t)
    R131_prob = measure(R131) # +,- prob(t1 =+),Prob(t1=-)


    R132 = ramsey(angle,w1,w2,l,0,d,d,notgate*R131_prob[2],r1t21,r1t22,t)
    R132_prob = measure(R132) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R133 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,notgate*R132_prob[2],r1t31,r1t32,t)
    R133_prob = measure(R133) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R134 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R132_prob[3],r1t31,r1t32,t)
    R134_prob = measure(R134) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R135 = ramsey(angle,w1,w2,l,0,d,d,R131_prob[3],r1t21,r1t22,t)
    R135_prob = measure(R135)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R136 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d, notgate*R135_prob[2],r1t31,r1t32,t)
    R136_prob = measure(R136) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R137 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R135_prob[3],r1t31,r1t32,t)
    R137_prob = measure(R137) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11R A12I A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    R141 = ramsey(angle,w1,w2,0,l,d,d,initial_state,r1t11,r1t12,t)
    R141_prob = measure(R141) # +,- prob(t1 =+),Prob(t1=-)


    R142 = ramsey(angle+pi/2,w1,w2,l,0,d,d,notgate*R141_prob[2],r1t21,r1t22,t)
    R142_prob = measure(R142) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R143 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,notgate*R142_prob[2],r1t31,r1t32,t)
    R143_prob = measure(R143) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R144 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R142_prob[3],r1t31,r1t32,t)
    R144_prob = measure(R144) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R145 = ramsey(angle+pi/2,w1,w2,l,0,d,d,R141_prob[3],r1t21,r1t22,t)
    R145_prob = measure(R145)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R146 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d, notgate*R145_prob[2],r1t31,r1t32,t)
    R146_prob = measure(R146) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R147 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R145_prob[3],r1t31,r1t32,t)
    R147_prob = measure(R147) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    #three time corellation
    expectterm1= (R111_prob[0]*R112_prob[0]*(R113_prob[0]-R113_prob[1])) + (R111_prob[0]*R112_prob[1]*(-R114_prob[0]+R114_prob[1])) + (R111_prob[1]*R115_prob[0]*(-R116_prob[0]+R116_prob[1]))+(R111_prob[1]*R115_prob[1]*(R117_prob[0]-R117_prob[1]))
    expectterm2=(R121_prob[0]*R122_prob[0]*(R123_prob[0]-R123_prob[1])) + (R121_prob[0]*R122_prob[1]*(-R124_prob[0]+R124_prob[1])) + (R121_prob[1]*R125_prob[0]*(-R126_prob[0]+R126_prob[1]))+(R121_prob[1]*R125_prob[1]*(R127_prob[0]-R127_prob[1]))
    expectterm3=(R131_prob[0]*R132_prob[0]*(R133_prob[0]-R133_prob[1])) + (R131_prob[0]*R132_prob[1]*(-R134_prob[0]+R134_prob[1])) + (R131_prob[1]*R135_prob[0]*(-R136_prob[0]+R136_prob[1]))+(R131_prob[1]*R135_prob[1]*(R137_prob[0]-R137_prob[1]))
    expectterm4=(R141_prob[0]*R142_prob[0]*(R143_prob[0]-R143_prob[1])) + (R141_prob[0]*R142_prob[1]*(-R144_prob[0]+R144_prob[1])) + (R141_prob[1]*R145_prob[0]*(-R146_prob[0]+R146_prob[1]))+(R141_prob[1]*R145_prob[1]*(R147_prob[0]-R147_prob[1]))

    total=  (expectterm1 - expectterm2 - expectterm3 - expectterm4)
    
    return total




In [95]:
def row3(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32):
    # Now we find the expectation value of the first term in the expresion (A11R A12R A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403
    print(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32)
    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R111 = ramsey(angle*2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,initial_state,r1t11,r1t12,t)
    R111_prob = measure(R111) # +,- prob(t1 =+),Prob(t1=-)


    R112 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,notgate*R111_prob[2],r1t21,r1t22,t)
    R112_prob = measure(R112) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R113 = ramsey(angle*2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,notgate*R112_prob[2],r1t31,r1t32,t)
    R113_prob = measure(R113) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R114 = ramsey(angle*2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,R112_prob[3],r1t31,r1t32,t)
    R114_prob = measure(R114) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R115 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R111_prob[3],r1t21,r1t22,t)
    R115_prob = measure(R115)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R116 = ramsey(angle*2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d, notgate*R115_prob[2],r1t31,r1t32,t)
    R116_prob = measure(R116) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R117 = ramsey(angle*2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,R115_prob[3],r1t31,r1t32,t)
    R117_prob = measure(R117) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11I A12I A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R121 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,initial_state,r1t11,r1t12,t) 
    R121_prob = measure(R121) # +,- prob(t1 =+),Prob(t1=-)


    R122 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,notgate*R121_prob[2],r1t21,r1t22,t)
    R122_prob = measure(R122) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R123 = ramsey(angle*2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,notgate*R122_prob[2],r1t31,r1t32,t)
    R123_prob = measure(R123) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R124 = ramsey(angle*2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,R122_prob[3],r1t31,r1t32,t)
    R124_prob = measure(R124) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R125 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R121_prob[3],r1t21,r1t22,t)
    R125_prob = measure(R125)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R126 = ramsey(angle*2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d, notgate*R125_prob[2],r1t31,r1t32,t)
    R126_prob = measure(R126) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R127 = ramsey(angle*2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,R125_prob[3],r1t31,r1t32,t)
    R127_prob = measure(R127) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    # Now we find the expectation value of the first term in the expresion (A11I A12R A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R131 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,initial_state,r1t11,r1t12,t)
    R131_prob = measure(R131) # +,- prob(t1 =+),Prob(t1=-)


    R132 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,notgate*R131_prob[2],r1t21,r1t22,t)
    R132_prob = measure(R132) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R133 = ramsey(angle*2+pi/2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,notgate*R132_prob[2],r1t31,r1t32,t)
    R133_prob = measure(R133) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R134 = ramsey(angle*2+pi/2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,R132_prob[3],r1t31,r1t32,t)
    R134_prob = measure(R134) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R135 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R131_prob[3],r1t21,r1t22,t)
    R135_prob = measure(R135)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R136 = ramsey(angle*2+pi/2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d, notgate*R135_prob[2],r1t31,r1t32,t)
    R136_prob = measure(R136) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R137 = ramsey(angle*2+pi/2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,R135_prob[3],r1t31,r1t32,t)
    R137_prob = measure(R137) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11R A12I A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    R141 = ramsey(angle*2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,initial_state,r1t11,r1t12,t)
    R141_prob = measure(R141) # +,- prob(t1 =+),Prob(t1=-)


    R142 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,notgate*R141_prob[2],r1t21,r1t22,t)
    R142_prob = measure(R142) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R143 = ramsey(angle*2+pi/2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,notgate*R142_prob[2],r1t31,r1t32,t)
    R143_prob = measure(R143) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R144 = ramsey(angle*2+pi/2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,R142_prob[3],r1t31,r1t32,t)
    R144_prob = measure(R144) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R145 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R141_prob[3],r1t21,r1t22,t)
    R145_prob = measure(R145)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R146 = ramsey(angle*2+pi/2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d, notgate*R145_prob[2],r1t31,r1t32,t)
    R146_prob = measure(R146) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R147 = ramsey(angle*2+pi/2,w1,w2,-np.exp(4*pi/3*1j)*l,-np.exp(4*pi/3*1j)*l,d,d,R145_prob[3],r1t31,r1t32,t)
    R147_prob = measure(R147) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    #three time corellation
    expectterm1= (R111_prob[0]*R112_prob[0]*(R113_prob[0]-R113_prob[1])) + (R111_prob[0]*R112_prob[1]*(-R114_prob[0]+R114_prob[1])) + (R111_prob[1]*R115_prob[0]*(-R116_prob[0]+R116_prob[1]))+(R111_prob[1]*R115_prob[1]*(R117_prob[0]-R117_prob[1]))
    expectterm2=(R121_prob[0]*R122_prob[0]*(R123_prob[0]-R123_prob[1])) + (R121_prob[0]*R122_prob[1]*(-R124_prob[0]+R124_prob[1])) + (R121_prob[1]*R125_prob[0]*(-R126_prob[0]+R126_prob[1]))+(R121_prob[1]*R125_prob[1]*(R127_prob[0]-R127_prob[1]))
    expectterm3=(R131_prob[0]*R132_prob[0]*(R133_prob[0]-R133_prob[1])) + (R131_prob[0]*R132_prob[1]*(-R134_prob[0]+R134_prob[1])) + (R131_prob[1]*R135_prob[0]*(-R136_prob[0]+R136_prob[1]))+(R131_prob[1]*R135_prob[1]*(R137_prob[0]-R137_prob[1]))
    expectterm4=(R141_prob[0]*R142_prob[0]*(R143_prob[0]-R143_prob[1])) + (R141_prob[0]*R142_prob[1]*(-R144_prob[0]+R144_prob[1])) + (R141_prob[1]*R145_prob[0]*(-R146_prob[0]+R146_prob[1]))+(R141_prob[1]*R145_prob[1]*(R147_prob[0]-R147_prob[1]))

    total=  (expectterm1 - expectterm2 - expectterm3 - expectterm4)
    
    return total




In [96]:
def column1(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32):
    # Now we find the expectation value of the first term in the expresion (A11R A12R A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403
    print(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32)
    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R111 = ramsey(angle,w1,w2,l,0,d,d,initial_state,r1t11,r1t12,t)
    R111_prob = measure(R111) # +,- prob(t1 =+),Prob(t1=-)


    R112 = ramsey(angle,w1,w2,0,np.exp(2*pi/3*1j)*l,d,d,notgate*R111_prob[2],r1t21,r1t22,t)
    R112_prob = measure(R112) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R113 = ramsey(angle*2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,notgate*R112_prob[2],r1t31,r1t32,t)
    R113_prob = measure(R113) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R114 = ramsey(angle*2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,R112_prob[3],r1t31,r1t32,t)
    R114_prob = measure(R114) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R115 = ramsey(angle,w1,w2,0,np.exp(2*pi/3*1j)*l,d,d,R111_prob[3],r1t21,r1t22,t)
    R115_prob = measure(R115)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R116 = ramsey(angle*2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d, notgate*R115_prob[2],r1t31,r1t32,t)
    R116_prob = measure(R116) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R117 = ramsey(angle*2,w1,w2,-l,np.exp(2*pi/3*1j)*l,d,d,R115_prob[3],r1t31,r1t32,t)
    R117_prob = measure(R117) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11I A12I A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R121 = ramsey(angle+pi/2,w1,w2,l,0,d,d,initial_state,r1t11,r1t12,t)
    R121_prob = measure(R121) # +,- prob(t1 =+),Prob(t1=-)

 
    R122 = ramsey(angle+pi/2,w1,w2,0,np.exp(2*pi/3*1j)*l,d,d,notgate*R121_prob[2],r1t21,r1t22,t)
    R122_prob = measure(R122) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R123 = ramsey(angle*2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,notgate*R122_prob[2],r1t31,r1t32,t)
    R123_prob = measure(R123) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R124 = ramsey(angle*2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,R122_prob[3],r1t31,r1t32,t)
    R124_prob = measure(R124) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R125 = ramsey(angle+pi/2,w1,w2,0,np.exp(2*pi/3*1j)*l,d,d,R121_prob[3],r1t21,r1t22,t)
    R125_prob = measure(R125)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R126 = ramsey(angle*2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d, notgate*R125_prob[2],r1t31,r1t32,t)
    R126_prob = measure(R126) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R127 = ramsey(angle*2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,R125_prob[3],r1t31,r1t32,t)
    R127_prob = measure(R127) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    # Now we find the expectation value of the first term in the expresion (A11I A12R A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R131 = ramsey(angle+pi/2,w1,w2,l,0,d,d,initial_state,r1t11,r1t12,t)
    R131_prob = measure(R131) # +,- prob(t1 =+),Prob(t1=-)


    R132 = ramsey(angle,w1,w2,0,np.exp(2*pi/3*1j)*l,d,d,notgate*R131_prob[2],r1t21,r1t22,t)
    R132_prob = measure(R132) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R133 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,notgate*R132_prob[2],r1t31,r1t32,t)
    R133_prob = measure(R133) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R134 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,R132_prob[3],r1t31,r1t32,t)
    R134_prob = measure(R134) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R135 = ramsey(angle,w1,w2,0,np.exp(2*pi/3*1j)*l,d,d,R131_prob[3],r1t21,r1t22,t)
    R135_prob = measure(R135)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R136 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d, notgate*R135_prob[2],r1t31,r1t32,t)
    R136_prob = measure(R136) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R137 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,R135_prob[3],r1t31,r1t32,t)
    R137_prob = measure(R137) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11R A12I A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    R141 = ramsey(angle,w1,w2,l,0,d,d,initial_state,r1t11,r1t12,t)
    R141_prob = measure(R141) # +,- prob(t1 =+),Prob(t1=-)


    R142 = ramsey(angle+pi/2,w1,w2,0,np.exp(2*pi/3*1j)*l,d,d,notgate*R141_prob[2],r1t21,r1t22,t)
    R142_prob = measure(R142) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R143 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,notgate*R142_prob[2],r1t31,r1t32,t)
    R143_prob = measure(R143) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R144 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,R142_prob[3],r1t31,r1t32,t)
    R144_prob = measure(R144) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R145 = ramsey(angle+pi/2,w1,w2,0,np.exp(2*pi/3*1j)*l,d,d,R141_prob[3],r1t21,r1t22,t)
    R145_prob = measure(R145)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R146 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d, notgate*R145_prob[2],r1t31,r1t32,t)
    R146_prob = measure(R146) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R147 = ramsey(angle*2+pi/2,w1,w2,-l,-np.exp(2*pi/3*1j)*l,d,d,R145_prob[3],r1t31,r1t32,t)
    R147_prob = measure(R147) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    #three time corellation
    expectterm1= (R111_prob[0]*R112_prob[0]*(R113_prob[0]-R113_prob[1])) + (R111_prob[0]*R112_prob[1]*(-R114_prob[0]+R114_prob[1])) + (R111_prob[1]*R115_prob[0]*(-R116_prob[0]+R116_prob[1]))+(R111_prob[1]*R115_prob[1]*(R117_prob[0]-R117_prob[1]))
    expectterm2=(R121_prob[0]*R122_prob[0]*(R123_prob[0]-R123_prob[1])) + (R121_prob[0]*R122_prob[1]*(-R124_prob[0]+R124_prob[1])) + (R121_prob[1]*R125_prob[0]*(-R126_prob[0]+R126_prob[1]))+(R121_prob[1]*R125_prob[1]*(R127_prob[0]-R127_prob[1]))
    expectterm3=(R131_prob[0]*R132_prob[0]*(R133_prob[0]-R133_prob[1])) + (R131_prob[0]*R132_prob[1]*(-R134_prob[0]+R134_prob[1])) + (R131_prob[1]*R135_prob[0]*(-R136_prob[0]+R136_prob[1]))+(R131_prob[1]*R135_prob[1]*(R137_prob[0]-R137_prob[1]))
    expectterm4=(R141_prob[0]*R142_prob[0]*(R143_prob[0]-R143_prob[1])) + (R141_prob[0]*R142_prob[1]*(-R144_prob[0]+R144_prob[1])) + (R141_prob[1]*R145_prob[0]*(-R146_prob[0]+R146_prob[1]))+(R141_prob[1]*R145_prob[1]*(R147_prob[0]-R147_prob[1]))

    total=  (expectterm1 - expectterm2 - expectterm3 - expectterm4)
    
    return total




In [97]:
-np.exp(2*pi/3*1j)*l,-l

(np.complex128(7853.981633974479-13603.495231756635j), -15707.963267948966)

In [98]:
def column2(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32):
    # Now we find the expectation value of the first term in the expresion (A11R A12R A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403
    print(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32)
    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R111 = ramsey(angle,w1,w2,0,l,d,d,initial_state,r1t11,r1t12,t)
    R111_prob = measure(R111) # +,- prob(t1 =+),Prob(t1=-)


    R112 = ramsey(angle,w1,w2,np.exp(2*pi/3*1j)*l,0,d,d,notgate*R111_prob[2],r1t21,r1t22,t)
    R112_prob = measure(R112) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R113 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,notgate*R112_prob[2],r1t31,r1t32,t)
    R113_prob = measure(R113) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R114 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R112_prob[3],r1t31,r1t32,t)
    R114_prob = measure(R114) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R115 = ramsey(angle,w1,w2,np.exp(2*pi/3*1j)*l,0,d,d,R111_prob[3],r1t21,r1t22,t)
    R115_prob = measure(R115)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R116 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d, notgate*R115_prob[2],r1t31,r1t32,t)
    R116_prob = measure(R116) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R117 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R115_prob[3],r1t31,r1t32,t)
    R117_prob = measure(R117) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11I A12I A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R121 = ramsey(angle+pi/2,w1,w2,0,l,d,d,initial_state,r1t11,r1t12,t)
    R121_prob = measure(R121) # +,- prob(t1 =+),Prob(t1=-)


    R122 = ramsey(angle+pi/2,w1,w2,np.exp(2*pi/3*1j)*l,0,d,d,notgate*R121_prob[2],r1t21,r1t22,t)
    R122_prob = measure(R122) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R123 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,notgate*R122_prob[2],r1t31,r1t32,t)
    R123_prob = measure(R123) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R124 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R122_prob[3],r1t31,r1t32,t)
    R124_prob = measure(R124) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R125 = ramsey(angle+pi/2,w1,w2,np.exp(2*pi/3*1j)*l,0,d,d,R121_prob[3],r1t21,r1t22,t)
    R125_prob = measure(R125)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R126 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d, notgate*R125_prob[2],r1t31,r1t32,t)
    R126_prob = measure(R126) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R127 = ramsey(angle*2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R125_prob[3],r1t31,r1t32,t)
    R127_prob = measure(R127) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    # Now we find the expectation value of the first term in the expresion (A11I A12R A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R131 = ramsey(angle+pi/2,w1,w2,0,l,d,d,initial_state,r1t11,r1t12,t)
    R131_prob = measure(R131) # +,- prob(t1 =+),Prob(t1=-)


    R132 = ramsey(angle,w1,w2,np.exp(2*pi/3*1j)*l,0,d,d,notgate*R131_prob[2],r1t21,r1t22,t)
    R132_prob = measure(R132) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R133 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,notgate*R132_prob[2],r1t31,r1t32,t)
    R133_prob = measure(R133) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R134 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R132_prob[3],r1t31,r1t32,t)
    R134_prob = measure(R134) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R135 = ramsey(angle,w1,w2,np.exp(2*pi/3*1j)*l,0,d,d,R131_prob[3],r1t21,r1t22,t)
    R135_prob = measure(R135)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R136 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d, notgate*R135_prob[2],r1t31,r1t32,t)
    R136_prob = measure(R136) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R137 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R135_prob[3],r1t31,r1t32,t)
    R137_prob = measure(R137) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11R A12I A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    R141 = ramsey(angle,w1,w2,0,l,d,d,initial_state,r1t11,r1t12,t)
    R141_prob = measure(R141) # +,- prob(t1 =+),Prob(t1=-)


    R142 = ramsey(angle+pi/2,w1,w2,np.exp(2*pi/3*1j)*l,0,d,d,notgate*R141_prob[2],r1t21,r1t22,t)
    R142_prob = measure(R142) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R143 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,notgate*R142_prob[2],r1t31,r1t32,t)
    R143_prob = measure(R143) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R144 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R142_prob[3],r1t31,r1t32,t)
    R144_prob = measure(R144) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R145 = ramsey(angle+pi/2,w1,w2,np.exp(2*pi/3*1j)*l,0,d,d,R141_prob[3],r1t21,r1t22,t)
    R145_prob = measure(R145)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R146 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d, notgate*R145_prob[2],r1t31,r1t32,t)
    R146_prob = measure(R146) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R147 = ramsey(angle*2+pi/2,w1,w2,-np.exp(2*pi/3*1j)*l,-l,d,d,R145_prob[3],r1t31,r1t32,t)
    R147_prob = measure(R147) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    #three time corellation
    expectterm1= (R111_prob[0]*R112_prob[0]*(R113_prob[0]-R113_prob[1])) + (R111_prob[0]*R112_prob[1]*(-R114_prob[0]+R114_prob[1])) + (R111_prob[1]*R115_prob[0]*(-R116_prob[0]+R116_prob[1]))+(R111_prob[1]*R115_prob[1]*(R117_prob[0]-R117_prob[1]))
    expectterm2=(R121_prob[0]*R122_prob[0]*(R123_prob[0]-R123_prob[1])) + (R121_prob[0]*R122_prob[1]*(-R124_prob[0]+R124_prob[1])) + (R121_prob[1]*R125_prob[0]*(-R126_prob[0]+R126_prob[1]))+(R121_prob[1]*R125_prob[1]*(R127_prob[0]-R127_prob[1]))
    expectterm3=(R131_prob[0]*R132_prob[0]*(R133_prob[0]-R133_prob[1])) + (R131_prob[0]*R132_prob[1]*(-R134_prob[0]+R134_prob[1])) + (R131_prob[1]*R135_prob[0]*(-R136_prob[0]+R136_prob[1]))+(R131_prob[1]*R135_prob[1]*(R137_prob[0]-R137_prob[1]))
    expectterm4=(R141_prob[0]*R142_prob[0]*(R143_prob[0]-R143_prob[1])) + (R141_prob[0]*R142_prob[1]*(-R144_prob[0]+R144_prob[1])) + (R141_prob[1]*R145_prob[0]*(-R146_prob[0]+R146_prob[1]))+(R141_prob[1]*R145_prob[1]*(R147_prob[0]-R147_prob[1]))

    total=  (expectterm1 - expectterm2 - expectterm3 - expectterm4)
    
    return total




In [99]:
# Define the coherent states for the oscillator
coherent_state1 = coherent(N, alpha1)  # Coherent state for the oscillator here at vacuum
coherent_state2 = coherent(N,alpha2)

# Combine the states using the tensor product (qubit, mode 1, mode 2)
initial_state = tensor(basis(2,0),coherent_state1,coherent_state2)

def column3(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32):
    # Now we find the expectation value of the first term in the expresion (A11R A12R A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403
    print(r1t11,r1t12,r1t21,r1t22,r1t31,r1t32)
    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R111 = ramsey(angle*2,w1,w2,l,l,d,d,initial_state,r1t11,r1t12,t)
    R111_prob = measure(R111) # +,- prob(t1 =+),Prob(t1=-)


    R112 = ramsey(angle*2,w1,w2,l,l,d,d,notgate*R111_prob[2],r1t21,r1t22,t)
    R112_prob = measure(R112) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R113 = ramsey(angle*2,w1,w2,l,l,d,d,notgate*R112_prob[2],r1t31,r1t32,t)
    R113_prob = measure(R113) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R114 = ramsey(angle*2,w1,w2,l,l,d,d,R112_prob[3],r1t31,r1t32,t)
    R114_prob = measure(R114) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R115 = ramsey(angle*2,w1,w2,l,l,d,d,R111_prob[3],r1t21,r1t22,t)
    R115_prob = measure(R115)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R116 = ramsey(angle*2,w1,w2,l,l,d,d, notgate*R115_prob[2],r1t31,r1t32,t)
    R116_prob = measure(R116) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R117 = ramsey(angle*2,w1,w2,l,l,d,d,R115_prob[3],r1t31,r1t32,t)
    R117_prob = measure(R117) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11I A12I A12R) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R121 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,initial_state,r1t11,r1t12,t)
    R121_prob = measure(R121) # +,- prob(t1 =+),Prob(t1=-)


    R122 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,notgate*R121_prob[2],r1t21,r1t22,t)
    R122_prob = measure(R122) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R123 = ramsey(angle*2,w1,w2,l,l,d,d,notgate*R122_prob[2],r1t31,r1t32,t)
    R123_prob = measure(R123) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R124 = ramsey(angle*2,w1,w2,l,l,d,d,R122_prob[3],r1t31,r1t32,t)
    R124_prob = measure(R124) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R125 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R121_prob[3],r1t21,r1t22,t)
    R125_prob = measure(R125)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R126 = ramsey(angle*2,w1,w2,l,l,d,d, notgate*R125_prob[2],r1t31,r1t32,t)
    R126_prob = measure(R126) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R127 = ramsey(angle*2,w1,w2,l,l,d,d,R125_prob[3],r1t31,r1t32,t)
    R127_prob = measure(R127) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    # Now we find the expectation value of the first term in the expresion (A11I A12R A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    angle = (l**2/d**2)*(d*t - np.sin(d*t))


    R131 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,initial_state,r1t11,r1t12,t)
    R131_prob = measure(R131) # +,- prob(t1 =+),Prob(t1=-)


    R132 = ramsey(angle*2,w1,w2,l,l,d,d,notgate*R131_prob[2],r1t21,r1t22,t)
    R132_prob = measure(R132) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R133 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,notgate*R132_prob[2],r1t31,r1t32,t)
    R133_prob = measure(R133) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R134 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R132_prob[3],r1t31,r1t32,t)
    R134_prob = measure(R134) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R135 = ramsey(angle*2,w1,w2,l,l,d,d,R131_prob[3],r1t21,r1t22,t)
    R135_prob = measure(R135)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R136 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d, notgate*R135_prob[2],r1t31,r1t32,t)
    R136_prob = measure(R136) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R137 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R135_prob[3],r1t31,r1t32,t)
    R137_prob = measure(R137) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)


    # Now we find the expectation value of the first term in the expresion (A11R A12I A12I) in Eq. 10 of the paper 10.1103/PhysRevLett.114.250403

    R141 = ramsey(angle*2,w1,w2,l,l,d,d,initial_state,r1t11,r1t12,t)
    R141_prob = measure(R141) # +,- prob(t1 =+),Prob(t1=-)


    R142 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,notgate*R141_prob[2],r1t21,r1t22,t)
    R142_prob = measure(R142) # prob(t2=+|t1=+) prob(t2=-|t1=+)

    R143 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,notgate*R142_prob[2],r1t31,r1t32,t)
    R143_prob = measure(R143) # prob(t3=+|t2=+|t1=+) prob(t3=-|t2=+|t1=+)

    R144 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R142_prob[3],r1t31,r1t32,t)
    R144_prob = measure(R144) #  prob(t3=+|t2=-|t1=+) prob(t3=-|t2=-|t1=+)

    R145 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R141_prob[3],r1t21,r1t22,t)
    R145_prob = measure(R145)  #prob(t2=+|t1=-) prob(t2=-|t1=-)

    R146 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d, notgate*R145_prob[2],r1t31,r1t32,t)
    R146_prob = measure(R146) # prob(t3=+|t2=+|t1=-) prob(t3=-|t2=+|t1=-)

    R147 = ramsey(angle*2+pi/2,w1,w2,l,l,d,d,R145_prob[3],r1t31,r1t32,t)
    R147_prob = measure(R147) # prob(t3=+|t2=-|t1=-) prob(t3=-|t2=-|t1=-)

    #three time corellation
    expectterm1= (R111_prob[0]*R112_prob[0]*(R113_prob[0]-R113_prob[1])) + (R111_prob[0]*R112_prob[1]*(-R114_prob[0]+R114_prob[1])) + (R111_prob[1]*R115_prob[0]*(-R116_prob[0]+R116_prob[1]))+(R111_prob[1]*R115_prob[1]*(R117_prob[0]-R117_prob[1]))
    expectterm2=(R121_prob[0]*R122_prob[0]*(R123_prob[0]-R123_prob[1])) + (R121_prob[0]*R122_prob[1]*(-R124_prob[0]+R124_prob[1])) + (R121_prob[1]*R125_prob[0]*(-R126_prob[0]+R126_prob[1]))+(R121_prob[1]*R125_prob[1]*(R127_prob[0]-R127_prob[1]))
    expectterm3=(R131_prob[0]*R132_prob[0]*(R133_prob[0]-R133_prob[1])) + (R131_prob[0]*R132_prob[1]*(-R134_prob[0]+R134_prob[1])) + (R131_prob[1]*R135_prob[0]*(-R136_prob[0]+R136_prob[1]))+(R131_prob[1]*R135_prob[1]*(R137_prob[0]-R137_prob[1]))
    expectterm4=(R141_prob[0]*R142_prob[0]*(R143_prob[0]-R143_prob[1])) + (R141_prob[0]*R142_prob[1]*(-R144_prob[0]+R144_prob[1])) + (R141_prob[1]*R145_prob[0]*(-R146_prob[0]+R146_prob[1]))+(R141_prob[1]*R145_prob[1]*(R147_prob[0]-R147_prob[1]))

    total=  (expectterm1 - expectterm2 - expectterm3 - expectterm4)
    
    return total




In [104]:
R1 = row1(r1t1,r1t1,r1t2,r1t2,r1t3,r1t3)

0 0 0.0008571428571428572 0.0008571428571428572 0.0018571428571428573 0.0018571428571428573


In [105]:
R1

0.9999984640037898

In [102]:
R2 = row2(r2t1,r2t1,r2t2,r2t2,r2t3,r2t3)

9.523809523809524e-05 9.523809523809524e-05 0.0009523809523809525 0.0009523809523809525 0.0019523809523809524 0.0019523809523809524


In [103]:
R2

0.9999984640048587

In [19]:
R3 = row3(r3t1,r3t1,r3t2,r3t2,r3t3,r3t3)

0 0 0.0008571428571428572 0.0008571428571428572 0.0017142857142857144 0.0017142857142857144


In [20]:
C1 = column1(c1t1,c1t1,c1t2,c1t2,c1t3,c1t3)

0 0 0.0014285714285714286 0.0014285714285714286 0.002857142857142857 0.002857142857142857


In [21]:
C1

0.9999985697714882

In [22]:
C2 = column2(c1t1,c1t1,c1t2,c1t2,c1t3,c1t3)

0 0 0.0014285714285714286 0.0014285714285714286 0.002857142857142857 0.002857142857142857


In [23]:
C2

0.9999984640039504

In [24]:
C3 = column3(c3t1,c3t1,c3t2,c3t2,c3t3,c3t3)

0.00014285714285714287 0.00014285714285714287 0.0010952380952380953 0.0010952380952380953 0.0023333333333333335 0.0023333333333333335


In [25]:
R1 + R2 + R3 + C1 + C2 - C3

5.999491793524647